In [1]:
import psycopg2
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Connect to Database

In [3]:
#Connect to PgAdmin
engine = psycopg2.connect(
    database="project-data",
    user = "postgres",
    password = "Post23!Fnord",
    host = "happinessproject.cqkpnj5dcmou.us-east-1.rds.amazonaws.com",
    port = '5432'
    )

cursor = engine.cursor()


In [4]:
#Pull combined data table
query = "SELECT * from combined_data"

df = pd.read_sql(query, con=engine)

In [5]:
df.head()

,country_name,region,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,...,ladder_2011,ladder_2012,ladder_2013,ladder_2014,ladder_2015,ladder_2016,ladder_2017,ladder_2018,ladder_2019,ladder_2020
0,Afghanistan,South Asia,2.523,0.038,2.596,2.449,7.695,0.463,52.493,0.382,...,3.832,3.783,3.572,3.131,3.983,4.220,2.662,2.694,2.375,NaN
1,Albania,Central and Eastern Europe,5.117,0.059,5.234,5.001,9.520,0.697,68.999,0.785,...,5.867,5.510,4.551,4.814,4.607,4.511,4.640,5.004,4.995,5.365
2,Algeria,Middle East and North Africa,4.887,0.053,4.991,4.783,9.342,0.802,66.005,0.480,...,5.317,5.605,NaN,6.355,NaN,5.341,5.249,5.043,4.745,NaN
3,Argentina,Latin America and Caribbean,5.929,0.056,6.040,5.819,9.962,0.898,69.000,0.828,...,6.776,6.468,6.582,6.671,6.697,6.427,6.039,5.793,6.086,5.901
4,Armenia,Commonwealth of Independent States,5.283,0.058,5.397,5.168,9.487,0.799,67.055,0.825,...,4.260,4.320,4.277,4.453,4.348,4.325,4.288,5.062,5.488,NaN


# Clean master table for Machine Learning

In [6]:
#column names
list(df)

['country_name',
 'region',
 'ladder_score',
 'stand_err',
 'up_whisker',
 'low_whisker',
 'log_gdp_per_cap',
 'social_support',
 'healty_life_exp',
 'freedom',
 'generosity',
 'percept_corrupt',
 'ladder_score_distopia',
 'expby_log_gdp_per_cap',
 'expby_social_support',
 'expby_healty_life_exp',
 'expby_freedom',
 'expby_generosity',
 'expby_percept_corrupt',
 'distopia_plus_resid',
 'ac_both_sexes',
 'ac_male',
 'ac_female',
 'population_2021',
 'covid_total_cases',
 'covid_new_cases',
 'covid_total_deaths',
 'covid_new_deaths',
 'covid_total_recovered',
 'covid_new_recovered',
 'covid_active_cases',
 'covid_serious_critical',
 'covid_cases_per_mil',
 'covid_deaths_per_mil',
 'covid_total_tests',
 'covid_tests_per_mil',
 'covid_who_region',
 'population_2020',
 'land_area_skm',
 'density_skm',
 'meat_consumption',
 'median_age',
 'screen_time_avg',
 'suicide_rate',
 'ladder_2010',
 'ladder_2011',
 'ladder_2012',
 'ladder_2013',
 'ladder_2014',
 'ladder_2015',
 'ladder_2016',
 'ladde

In [7]:
df2 = df.filter(['ladder_score', 'freedom','healty_life_exp', 'log_gdp_per_cap', 'meat_consumption',
                 'percept_corrupt', 'social_support', 'covid_tests_per_mil', 'ac_female', 'generosity', 'covid_deaths_per_mil', 'covid_total_tests', 'covid_active_cases'], axis=1)

In [8]:
# view total null values for columns
df2.isna().sum()

ladder_score             0
freedom                  0
healty_life_exp          0
log_gdp_per_cap          0
meat_consumption         9
percept_corrupt          0
social_support           0
covid_tests_per_mil     17
ac_female                8
generosity               0
covid_deaths_per_mil    10
covid_total_tests       17
covid_active_cases      11
dtype: int64

In [9]:
#compare NA values to dataset row count
len(df2)

149

In [10]:
#drop all rows containing NA values
df2 = df2.dropna()

In [11]:
# view total null values for columns
df2.isna().sum()

ladder_score            0
freedom                 0
healty_life_exp         0
log_gdp_per_cap         0
meat_consumption        0
percept_corrupt         0
social_support          0
covid_tests_per_mil     0
ac_female               0
generosity              0
covid_deaths_per_mil    0
covid_total_tests       0
covid_active_cases      0
dtype: int64

In [12]:
#length of final dataset for analysis 
len(df2)

120

# Random Forest: Find the 10 variables that impact happiness scores the most 

In [13]:
# Create columns and target variables
columns = [
    'ladder_score', 'freedom','healty_life_exp', 'log_gdp_per_cap', 'meat_consumption',
                 'percept_corrupt', 'social_support', 'covid_tests_per_mil', 'ac_female', 
    'generosity', 'covid_deaths_per_mil', 'covid_total_tests', 'covid_active_cases'
]

target = ["ladder_score"]

In [14]:
#Convert variable type for target to non-continuous dtype
df2 = df2.astype({'ladder_score':'int'})

df2.head()

,ladder_score,freedom,healty_life_exp,log_gdp_per_cap,meat_consumption,percept_corrupt,social_support,covid_tests_per_mil,ac_female,generosity,covid_deaths_per_mil,covid_total_tests,covid_active_cases
0,2,0.382,52.493,7.695,17.3,0.924,0.463,2317.0,0.003,-0.102,33.0,90396.0,9758.0
3,5,0.828,69.000,9.962,79.7,0.834,0.898,17564.0,4.640,-0.182,94.0,794544.0,124092.0
4,5,0.825,67.055,9.487,27.7,0.629,0.799,57898.0,2.140,-0.168,260.0,171600.0,7491.0
5,7,0.914,73.900,10.796,108.2,0.442,0.940,181419.0,5.110,0.159,10.0,4631419.0,8694.0
6,7,0.908,73.300,10.906,94.1,0.481,0.934,104008.0,5.760,0.042,80.0,937275.0,1381.0


In [15]:
# Create our features
X = pd.get_dummies(df2.drop(columns="ladder_score"))


# Create our target
y = df2["ladder_score"]

In [16]:
X.describe()

,freedom,healty_life_exp,log_gdp_per_cap,meat_consumption,percept_corrupt,social_support,covid_tests_per_mil,ac_female,generosity,covid_deaths_per_mil,covid_total_tests,covid_active_cases
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,1.200000e+02,1.200000e+02
mean,0.796400,65.162900,9.477950,45.711667,0.730467,0.816300,74698.991667,2.595283,-0.024800,93.894833,1.972836e+06,4.659779e+04
std,0.111811,6.540754,1.147311,34.718099,0.180942,0.116285,126572.644846,1.994211,0.154735,150.569150,6.741017e+06,2.267261e+05
min,0.382000,48.700000,6.635000,3.100000,0.082000,0.463000,4.000000,0.000000,-0.288000,0.080000,1.200000e+02,0.000000e+00
25%,0.729750,60.427750,8.572750,17.125000,0.679750,0.759000,8405.750000,0.820000,-0.135500,6.000000,1.247078e+05,5.567500e+02
50%,0.810500,66.700500,9.584500,36.600000,0.791500,0.834000,35939.000000,2.160000,-0.050000,29.500000,3.402930e+05,3.583500e+03
75%,0.882500,69.534250,10.391750,67.900000,0.845500,0.905250,92173.000000,4.367500,0.064250,110.000000,1.379390e+06,1.339850e+04
max,0.970000,76.953000,11.647000,145.900000,0.939000,0.983000,995282.000000,6.340000,0.542000,850.000000,6.313960e+07,2.292707e+06


In [17]:
# Check the balance of our target values
y.value_counts()

5    38
6    30
4    28
7    14
3     9
2     1
Name: ladder_score, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({4: 21, 5: 31, 6: 21, 3: 5, 7: 11, 2: 1})

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [20]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.4

In [21]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[0, 2, 2, 0, 0],
       [0, 3, 4, 0, 0],
       [0, 2, 4, 1, 0],
       [0, 0, 5, 3, 1],
       [0, 0, 0, 1, 2]], dtype=int64)

In [22]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          3       0.00      0.00      1.00      0.00      0.00      0.00         4
          4       0.43      0.43      0.83      0.43      0.60      0.34         7
          5       0.27      0.57      0.52      0.36      0.55      0.30         7
          6       0.60      0.33      0.90      0.43      0.55      0.28         9
          7       0.67      0.67      0.96      0.67      0.80      0.62         3

avg / total       0.41      0.40      0.82      0.38      0.51      0.30        30



In [23]:
#export DataFrame to CSV file
#df2.to_csv('happiness_top12_df.csv', index=False)